<a href="https://colab.research.google.com/github/TheAnders121/TestRepo/blob/main/ProjektKode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import libraries**

In [3]:
import os
import datetime
import time

from datetime import datetime, timedelta

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import RobustScaler
import json
import requests

**Configurations**

In [2]:
MAX_EPOCHS = 20

multi_val_performance = {}
multi_performance = {}

inputTransformer = RobustScaler()
outputTransformer = RobustScaler()

#Paths
model_path = '/content/saved_model/my_model'
rData_path = 'data.csv'
wData_path = ''
plot_path = ''

#Model columns
direct = 'DirectSunPower kW/m²'
diffused = 'DiffuseSunPower kW/m²'
sunAltitude = 'SunAltitude °'
inputColumn = diffused
outputColumn = sunAltitude

#Model construction
steps = 24
inputColumns = 2
inputShape = [steps, inputColumns]

#Model compilation, fitting and optimization
optimizationAlgorithm = tf.keras.optimizers.Adam()

#API paths 
POST = 'somePath'
GET = 'somePath'

#Scheduler
hours = 1


**Data windowing**

**Split data**

In [ ]:
def splitData(data):
  n = len(data)
  train = data[0:int(n*0.9)] #first 90%
  test = data[int(n*0.9):] #left over 10%
  return [train, test]

**Create dataset**

In [ ]:
def createDatasets(input, output, timeSteps):
  inputList, outputList = [], []
  for i in range(len(input) - timeSteps):
    val = input.iloc[i: (i + timeSteps)].to_numpy()
    inputList.append(val)
    outputList.append(output.iloc[i + timeSteps])
  return np.array(inputList), np.array(outputList)

**Reshape data**

In [ ]:
def reshapeData(data, steps):
  input, output = createDatasets(data, data[outputColumn], steps)
  return [input, output]

**Transform data**

In [ ]:
def transformData(data):
  
  return

**Inverse transform data**

**To json string**

In [ ]:
def toJson(string):
  return json.dumps(string)

**Convert Json string**

In [ ]:
def toNormal(string):
  return json.loads(string)

**Write predictions to file**

In [ ]:
def writePredictions(string):
  # Append-adds at last
  file1 = open(wData_path, "w")  #write mode (overwrite previous predictions)
  file1.write(string)
  file1.close()

**Upload predictions**

In [ ]:
def uploadPredictions(): 
  api_url = "https://jsonplaceholder.typicode.com/todos"
  todo = {"userId": 1, "title": "Buy milk", "completed": False}
  response = requests.post(api_url, json=todo)
  return response.status_code

**Build model**

In [ ]:
def buildModel():
  model = keras.Sequential()
  model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
        units=128, 
        input_shape=inputShape
      )
    )
  )
  model.add(keras.layers.Dropout(rate=0.2))
  model.add(keras.layers.Dense(units=1))
  return model

**Compile and fit model**

In [ ]:
def compile(model, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=optimizationAlgorithm,
                metrics=[tf.keras.metrics.MeanAbsoluteError()])
  return model

**Fit model**

In [ ]:
def fit(model, input, output):
  history = model.fit(
    input, output,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
  )
  return history

**Save weights**

**Load weights**

**Save model**

In [ ]:
#model - model that is to be saved (needs to be a tf.keras.model object)
#path - under which path the model will be saved (and can be accessed)
def save_model(model, path):
  model.save(path)

**Load model**

In [ ]:
#path - under which path the model will be accessible
def load_model(path):
  new_model = tf.keras.models.load_model(path)

  return new_model

**Load data**

In [ ]:
#path - path to where the data is to be loaded from 
def load_data(path):
  dt = pd.read_csv(rData_path, sep=";",thousands='.', decimal=',') #read csv file with ';' as seperator and ',' as decimal marker (will be converted to '.')
  return dt

**Process data**

In [ ]:
#data - data that is to be processed in order for the model to read it
def process_data(data):
  # Convert to radians.
  data[sunAltitude] = data[sunAltitude]*np.pi / 180
  #data[[diffused, sunAltitude]].copy() #return only two relevant columns
  return data[[sunAltitude]].copy() #return only relevant column

**Predict**

In [ ]:
#data - 48 datapoints for the model to predict the next 24 datapoints
def make_prediction(model, data, plot):
  if plot == False:
    prediction = model.predict(data, verbose=0)
    return prediction
  #Code to return plot 


**Send prediction**

**Main function**

In [ ]:

print("Hello from main")

#while True:
  #Do work

#  time.sleep(3600 * hours)  # do work every x amount of hours




**Unzip file**

In [ ]:
!unzip /content/LSTM_Model.zip

Archive:  /content/LSTM_Model.zip
   creating: saved_model/my_model/
  inflating: saved_model/my_model/saved_model.pb  
  inflating: saved_model/my_model/keras_metadata.pb  
   creating: saved_model/my_model/assets/
   creating: saved_model/my_model/variables/
  inflating: saved_model/my_model/variables/variables.index  
  inflating: saved_model/my_model/variables/variables.data-00000-of-00001  


**Load test data**

In [ ]:
dt = pd.read_csv('data.csv', sep=";",thousands='.', decimal=',') #read csv file with ';' as seperator and ',' as decimal marker (will be converted to '.')

dt['SunAltitude °'] = dt['SunAltitude °']*np.pi / 180 #Get radians

diffuseSunData = dt[['DiffuseSunPower kW/m²','SunAltitude °']].copy()

n = len(diffuseSunData)
train_diffuseSunData = diffuseSunData[0:int(n*0.7)] #first 70%
val_diffuseSunData = diffuseSunData[int(n*0.7):int(n*0.9)] #following 20%
test_diffuseSunData = diffuseSunData[int(n*0.9):] #left over 10%

**Normalize data** 

In [ ]:
train_mean = diffuseSunData.mean()
train_var = diffuseSunData.var()

train_diffuseSunData = (train_diffuseSunData - train_mean) / train_var
val_diffuseSunData = (val_diffuseSunData - train_mean) / train_var
test_diffuseSunData = (test_diffuseSunData - train_mean) / train_var

**Denormalize**

In [ ]:
def denormalize(data):
  min_d = np.min(data)
  max_d = np.max(data)
  return (data - min_d) / (max_d - min_d)

**Model construction**

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(units=32, return_sequences=True, input_shape=(None, 1)))
model.add(tf.keras.layers.LSTM(units=32, return_sequences=True))
model.add(tf.keras.layers.LSTM(units=32))
model.add(tf.keras.layers.Dense(units=1))
model.summary()

model = compile(model)

model.summary()

print(train_diffuseSunData['DiffuseSunPower kW/m²'])

history = model.fit(train_diffuseSunData['SunAltitude °'],train_diffuseSunData['DiffuseSunPower kW/m²'], epochs=10)

model.evaluate(test_diffuseSunData['SunAltitude °'],test_diffuseSunData['DiffuseSunPower kW/m²'])

prediction = model.predict(train_diffuseSunData['SunAltitude °'])
print(prediction)

#print(denormalize(train_diffuseSunData['SunAltitude °']))

print(train_diffuseSunData['DiffuseSunPower kW/m²'])

from sklearn import metrics
metrics.mean_squared_error(train_diffuseSunData['DiffuseSunPower kW/m²'], prediction)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 32)          4352      
                                                                 
 lstm_1 (LSTM)               (None, None, 32)          8320      
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 21,025
Trainable params: 21,025
Non-trainable params: 0
_________________________________________________________________
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (No

78.00370806634253

**Test**

In [ ]:
model = load_model(model_path)

model.summary()

testInput = process_data(load_data(rData_path))
print(testInput[0:4])
testInput = testInput*np.pi / 180 #radians

print(testInput)
#test = np.array(testInput[10677:10677+48])
test = np.array(testInput[0:48])
test_mean = test.mean()
test_std = test.std()

test = (test - test_mean) / test_std

test = test.reshape(1,24,2)
print(f'TEST SHAPE: {test.shape}')

preds = model.predict(test, verbose=0)
print(f'PREDICTON: {preds}')


data = pd.read_csv(rData_path, sep=";",thousands='.', decimal=',') #read csv file with ';' as seperator and ',' as decimal marker (will be converted to '.')

trueData = data[[diffused]].copy()

#trueData = np.array(trueData[10677:10677+48])
trueData = np.array(trueData[0:48])
trueData = trueData.reshape(1,48,1)
trueData_mean = trueData.mean()
trueData_std = trueData.std()

trueData = (trueData - trueData_mean) / trueData_std

#show = load_data(rData_path)
#print(trueData)


#test_results = model.evaluate(preds, trueData, verbose=0)
#print(f" Accuracy: {test_results}")


data = pd.read_csv(rData_path, sep=";",thousands='.', decimal=',') #read csv file with ';' as seperator and ',' as decimal marker (will be converted to '.')

trueData = data[[diffused]].copy()
trueData = np.array(trueData)
trueData_mean = trueData.mean()
trueData_std = trueData.std()

trueData = (trueData - trueData_mean) / trueData_std

print(trueData[0:48])
#print(trueData[10677])
            


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                4480      
                                                                 
 dense_3 (Dense)             (None, 48)                1584      
                                                                 
 reshape_1 (Reshape)         (None, 24, 2)             0         
                                                                 
Total params: 6,064
Trainable params: 6,064
Non-trainable params: 0
_________________________________________________________________
   SunAltitude °
0       0.512254
1       0.510858
2       0.466701
3       0.385194
       SunAltitude °
0           0.008941
1           0.008916
2           0.008145
3           0.006723
4           0.004804
...              ...
15248       0.002751
15249       0.000688
15250      -0.001139
15251       0.000000
